In [17]:
import numpy as np
import os
import os.path as op
import pandas as pd

In [18]:
IMAGE_WIDTH = {5: 15, 20: 60, 60: 180}
IMAGE_HEIGHT = {5: 32, 20: 64, 60: 96}   

In [19]:
# trainX

images = {}

for year in np.arange(1993, 2000):
    images[str(year)] = np.memmap(op.join(os.getcwd()[:17] + 'MAFS6010Z_Project2/data/', f"monthly_20d/20d_month_has_vb_[20]_ma_{year}_images.dat"), dtype=np.uint8, mode='r').reshape(
                        (-1, IMAGE_HEIGHT[20], IMAGE_WIDTH[20]))


trainX = np.r_[images['1993'], images['1994'], images['1995'], images['1996'], images['1997'], images['1998'], 
               images['1999']]

In [20]:
# trainY

trainY = pd.Series()

for year in np.arange(1993, 2000):
    label_df = pd.read_feather(op.join(os.getcwd()[:17] + 'MAFS6010Z_Project2/data/', f"monthly_20d/20d_month_has_vb_[20]_ma_{year}_labels_w_delay.feather"))\
               ['Retx_20d_label']
    
    trainY = pd.concat([trainY, label_df])
    
trainY.reset_index(drop=True, inplace=True)

/tmp/ipykernel_21221/3373700141.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  trainY = pd.Series()


In [21]:
# testX

images = {}

for year in np.arange(2000, 2020):
    images[str(year)] = np.memmap(op.join(os.getcwd()[:17] + 'MAFS6010Z_Project2/data/', f"monthly_20d/20d_month_has_vb_[20]_ma_{year}_images.dat"), dtype=np.uint8, mode='r').reshape(
                        (-1, IMAGE_HEIGHT[20], IMAGE_WIDTH[20]))

testX = np.r_[images['2000'], images['2001'], images['2002'], images['2003'], images['2004'], images['2005'], 
               images['2006'], images['2007'], images['2008'], images['2009'], images['2010'], images['2011'], 
               images['2012'], images['2013'], images['2014'], images['2015'], images['2016'], images['2017'],
               images['2018'], images['2019']]

In [22]:
# testY

testY = pd.Series()

for year in np.arange(2000, 2020):
    label_df = pd.read_feather(op.join(os.getcwd()[:17] + 'MAFS6010Z_Project2/data/', f"monthly_20d/20d_month_has_vb_[20]_ma_{year}_labels_w_delay.feather"))\
               ['Retx_20d_label']
    
    testY = pd.concat([testY, label_df])

testY.reset_index(drop=True, inplace=True)

/tmp/ipykernel_21221/2517823635.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  testY = pd.Series()


In [23]:
import random
import torch
import torch.nn as nn
import torch.utils.data as Data
from torch.utils.data import TensorDataset

In [24]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [25]:
BATCH_SIZE = 128

In [26]:
X = trainX[128:].reshape((-1, IMAGE_HEIGHT[20], IMAGE_WIDTH[20]))
Y = trainY

twoIds = Y[Y == 2].index
Ids = [x for x in range(0, X.shape[0]) if x not in twoIds]

X = torch.Tensor(X[Ids])
Y = torch.Tensor(Y[Ids].values.reshape(1, -1))

ids = [x for x in range(0, X.shape[0])]
random.shuffle(ids)

In [27]:
X_train = X[ids[int(len(ids)*0.3):]]
train_y = Y[0][ids[int(len(ids)*0.3):]]

validate_X = X[ids[:int(len(ids)*0.3)]]
validate_y = Y[0][ids[:int(len(ids)*0.3)]]

In [28]:
test_X = testX.reshape((-1, IMAGE_HEIGHT[20], IMAGE_WIDTH[20]))
test_y = testY

twoIds = test_y[test_y == 2].index
Ids = [x for x in range(0, test_X.shape[0]) if x not in twoIds]

In [29]:
train_loader = Data.DataLoader(dataset=TensorDataset(X_train, train_y), \
                               batch_size=BATCH_SIZE, shuffle=True)
valid_loader = Data.DataLoader(dataset=TensorDataset(validate_X, validate_y), \
                               batch_size=BATCH_SIZE, shuffle=True)
test_loader = Data.DataLoader(dataset=TensorDataset(torch.Tensor(test_X[Ids]), torch.Tensor(test_y.loc[Ids].values.reshape(1, -1)[0])), \
                               batch_size=BATCH_SIZE, shuffle=True)

In [30]:
class CNN(nn.Module):
    
    def __init__(self, Dropout, BN, Xavier, Activation, Pool, Filter, Dilation, Stride):
        super(CNN, self).__init__()
        self.Dropout = Dropout
        self.BN = BN
        self.Xavier = Xavier
        self.Activation = Activation
        self.Pool = Pool
        self.Filter = Filter
        self.Dilation = Dilation
        self.Stride = Stride
        
        if self.BN:
            self.conv1 = nn.Sequential(
                nn.Conv2d(
                    in_channels=1,
                    out_channels=64,
                    kernel_size=self.Filter,
                    stride=self.Stride,
                    dilation=self.Dilation,
                    padding=(67,1)
                    ),
                nn.BatchNorm2d(64),
                nn.LeakyReLU(),
                nn.MaxPool2d(kernel_size=self.Pool),
            )

            self.conv2 = nn.Sequential( 
                nn.Conv2d(
                    in_channels=64,    
                    out_channels=128,  
                    kernel_size=self.Filter,
                    padding=(2,1)
                ),
                nn.BatchNorm2d(128),
                nn.LeakyReLU(),
                nn.MaxPool2d(kernel_size=self.Pool), 
            )

            self.conv3 = nn.Sequential(  
                nn.Conv2d(
                    in_channels=128,  
                    out_channels=256, 
                    kernel_size=self.Filter,
                    padding=(2,1)
                ),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(),
                nn.MaxPool2d(kernel_size=self.Pool), 
            )
        
        if self.BN == False:
            self.conv1 = nn.Sequential(
                nn.Conv2d(
                    in_channels=1,
                    out_channels=64,
                    kernel_size=self.Filter,
                    stride=self.Stride,
                    dilation=self.Dilation,
                    padding=(67,1)
                    ),
                nn.LeakyReLU(),
                nn.MaxPool2d(kernel_size=self.Pool),
            )

            self.conv2 = nn.Sequential( 
                nn.Conv2d(
                    in_channels=64,    
                    out_channels=128,  
                    kernel_size=self.Filter,
                    padding=(2,1)
                ),
                nn.LeakyReLU(),
                nn.MaxPool2d(kernel_size=self.Pool), 
            )

            self.conv3 = nn.Sequential(  
                nn.Conv2d(
                    in_channels=128,  
                    out_channels=256, 
                    kernel_size=self.Filter,
                    padding=(2,1)
                ),
                nn.LeakyReLU(),
                nn.MaxPool2d(kernel_size=self.Pool), 
            )
        
        if self.Activation == 'LReLU':
            self.conv1 = nn.Sequential(
                nn.Conv2d(
                    in_channels=1,
                    out_channels=64,
                    kernel_size=self.Filter,
                    stride=self.Stride,
                    dilation=self.Dilation,
                    padding=(67,1)
                    ),
                nn.BatchNorm2d(64),
                nn.LeakyReLU(),
                nn.MaxPool2d(kernel_size=self.Pool),
            )

            self.conv2 = nn.Sequential( 
                nn.Conv2d(
                    in_channels=64,    
                    out_channels=128,  
                    kernel_size=self.Filter,
                    padding=(2,1)
                ),
                nn.BatchNorm2d(128),
                nn.LeakyReLU(),
                nn.MaxPool2d(kernel_size=self.Pool), 
            )

            self.conv3 = nn.Sequential(  
                nn.Conv2d(
                    in_channels=128,  
                    out_channels=256, 
                    kernel_size=self.Filter,
                    padding=(2,1)
                ),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(),
                nn.MaxPool2d(kernel_size=self.Pool), 
            )
        
        if self.Activation == 'ReLU':
            self.conv1 = nn.Sequential(
                nn.Conv2d(
                    in_channels=1,
                    out_channels=64,
                    kernel_size=self.Filter,
                    stride=self.Stride,
                    dilation=self.Dilation,
                    padding=(67,1)
                    ),
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=self.Pool),
            )

            self.conv2 = nn.Sequential( 
                nn.Conv2d(
                    in_channels=64,    
                    out_channels=128,  
                    kernel_size=self.Filter,
                    padding=(2,1)
                ),
                nn.BatchNorm2d(128),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=self.Pool), 
            )

            self.conv3 = nn.Sequential(  
                nn.Conv2d(
                    in_channels=128,  
                    out_channels=256, 
                    kernel_size=self.Filter,
                    padding=(2,1)
                ),
                nn.BatchNorm2d(256),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=self.Pool), 
            )
        
        if self.Xavier:
            for m in self.modules():
                if isinstance(m, (nn.Conv2d, nn.Linear)):
                    nn.init.xavier_normal_(m.weight)

        if self.Xavier == False:
            pass
        
        self.FC1 = nn.Linear(in_features=256*8*60, out_features=2)
        self.dropout = nn.Dropout(self.Dropout)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size()[0], -1)
        x = self.FC1(x)
        x = self.dropout(x)
        output = nn.functional.softmax(x,dim=1)
        
        return output

In [31]:
cnn = CNN(Dropout=0.5, BN=True, Xavier=True, Activation='LReLU', Pool=(2, 1), Filter=(5, 3), Dilation=(2,1), Stride=(3,1))
cnn = cnn.to(device)

optimizer = torch.optim.Adam(cnn.parameters(), lr=0.00001)
loss_func = nn.CrossEntropyLoss()

validate_loss_oldold = 10000000
validate_loss_old = 10000000

# Training and validating
for step,(b_x,b_y) in enumerate(train_loader):
    cnn.train()
    b_x = b_x.to(device)
    b_y = b_y.to(device)
    b_x = torch.unsqueeze(b_x,dim=1).float()
    output = cnn(b_x)
    loss = loss_func(output, b_y.long())
    optimizer.zero_grad()           # clear gradients for this training step
    loss.backward()                 # backpropagation, compute gradients
    optimizer.step()                # apply gradients
    
    if ((step % 220 == 0) & (step != 0)):
        loss_val = 0
        total_val = 0
        correct_val = 0
        cnn.eval()
        
        with torch.no_grad():
            for val_step,(v_x,v_y) in enumerate(valid_loader):
                v_x = v_x.to(device)
                v_y = v_y.to(device)
                v_x = torch.unsqueeze(v_x,dim=1).float()
                validate_output = cnn(v_x)
                validate_loss_new = loss_func(validate_output,v_y.long())
                loss_val += validate_loss_new.item()
                
                pred_y = torch.max(validate_output,1)[1].data
                total_val += v_y.size(0)
                correct_val += (pred_y == v_y).squeeze().sum()
            loss_val /= len(valid_loader)
                
        if (validate_loss_oldold <= validate_loss_old) and (validate_loss_old <= loss_val):
            
            print('TERMINAL')
            print('Accuracy :{:.3f}'.format(correct_val / total_val))
            print('oldold:{:.3f}'.format(validate_loss_oldold))
            print('old:{:.3f}'.format(validate_loss_old))
            print('new:{:.3f}'.format(loss_val))
            break
            
        print('oldold:{:.3f}'.format(validate_loss_oldold))
        print('old:{:.3f}'.format(validate_loss_old))

        validate_loss_oldold = validate_loss_old
        validate_loss_old = loss_val

        print('Validation times: ', step//220, '| validation loss: %.3f' % loss_val, '| validation accuracy: %.3f' % (correct_val / total_val))
        print('--------------------------------------------------')

oldold:10000000.000
old:10000000.000
Validation times:  1 | validation loss: 0.701 | validation accuracy: 0.508
--------------------------------------------------
oldold:10000000.000
old:0.701
Validation times:  2 | validation loss: 0.698 | validation accuracy: 0.500
--------------------------------------------------
oldold:0.701
old:0.698
Validation times:  3 | validation loss: 0.705 | validation accuracy: 0.511
--------------------------------------------------
oldold:0.698
old:0.705
Validation times:  4 | validation loss: 0.699 | validation accuracy: 0.500
--------------------------------------------------
oldold:0.705
old:0.699
Validation times:  5 | validation loss: 0.707 | validation accuracy: 0.512
--------------------------------------------------
oldold:0.699
old:0.707
Validation times:  6 | validation loss: 0.703 | validation accuracy: 0.512
--------------------------------------------------
oldold:0.707
old:0.703
Validation times:  7 | validation loss: 0.700 | validation acc

In [32]:
# Testing

total_val = 0
correct_val = 0

for step,(t_x,t_y) in enumerate(test_loader):
    
    cnn.eval()
    with torch.no_grad():
        
        t_x = t_x.to(device)
        t_y = t_y.to(device)
        t_x = torch.unsqueeze(t_x,dim=1).float()
        test_output = cnn(t_x)
        pred_y = torch.max(test_output,1)[1].data
        total_val += t_y.size(0)
        correct_val += (pred_y == t_y).squeeze().sum()
        
        validate_loss_new = loss_func(test_output,t_y.long())
        loss_val += validate_loss_new.item()
        
loss_val /= len(test_loader)

print('Accuracy:{:.3f}'.format(correct_val/ total_val))
print('Loss:{:.3f}'.format(loss_val))

Accuracy:0.506
Loss:0.700


In [20]:
# Robusted Test

def robustest(Dropout, BN, Xavier, Activation, Pool, Filter, Dilation, Stride):
    
    cnn = CNN(Dropout=Dropout, BN=BN, Xavier=Xavier, Activation=Activation, Pool=Pool, Filter=Filter, Dilation=Dilation, Stride=Stride)
    cnn = cnn.to(device)

    optimizer = torch.optim.Adam(cnn.parameters(), lr=0.00001)
    loss_func = nn.CrossEntropyLoss()

    validate_loss_oldold = 10000000
    validate_loss_old = 10000000

    for step,(b_x,b_y) in enumerate(train_loader):
        cnn.train()
        b_x = b_x.to(device)
        b_y = b_y.to(device)
        b_x = torch.unsqueeze(b_x,dim=1).float()
        output = cnn(b_x)
        loss = loss_func(output, b_y.long())
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients

        if ((step % 220 == 0) & (step != 0)):
            loss_val = 0
            total_val = 0
            correct_val = 0
            cnn.eval()

            with torch.no_grad():
                for val_step,(v_x,v_y) in enumerate(valid_loader):
                    v_x = v_x.to(device)
                    v_y = v_y.to(device)
                    v_x = torch.unsqueeze(v_x,dim=1).float()
                    validate_output = cnn(v_x)
                    validate_loss_new = loss_func(validate_output,v_y.long())
                    loss_val += validate_loss_new.item()

                    pred_y = torch.max(validate_output,1)[1].data
                    total_val += v_y.size(0)
                    correct_val += (pred_y == v_y).squeeze().sum()
                loss_val /= len(valid_loader)

            if (validate_loss_oldold <= validate_loss_old) and (validate_loss_old <= loss_val):

                print('TERMINAL')
                print('Accuracy :{:.3f}'.format(correct_val / total_val))
                print('oldold:{:.3f}'.format(validate_loss_oldold))
                print('old:{:.3f}'.format(validate_loss_old))
                print('new:{:.3f}'.format(loss_val))
                print('----------------------------------------')
                break

            validate_loss_oldold = validate_loss_old
            validate_loss_old = loss_val
            
    total_val = 0
    correct_val = 0

    for step,(t_x,t_y) in enumerate(test_loader):

        cnn.eval()
        with torch.no_grad():

            t_x = t_x.to(device)
            t_y = t_y.to(device)
            t_x = torch.unsqueeze(t_x,dim=1).float()
            test_output = cnn(t_x)
            pred_y = torch.max(test_output,1)[1].data
            total_val += t_y.size(0)
            correct_val += (pred_y == t_y).squeeze().sum()

            validate_loss_new = loss_func(test_output,t_y.long())
            loss_val += validate_loss_new.item()

    loss_val /= len(test_loader)

    print('Accuracy:{:.3f}'.format(correct_val / total_val))
    print('Loss:{:.3f}'.format(loss_val))


In [22]:
Dropout= [0, 0.25, 0.75]
Filter = [(3, 3), (7, 3)]
DilationStride = [[(2,1), (1,1)], [(1,1), (3,1)], [(1,1), (1,1)]]

In [13]:
for d in Dropout:
    print(d)
    robustest(d, BN=True, Xavier=True, Activation='LReLU', Pool=(2,1), Filter=(5,3), Dilation=(2,1), Stride=(3,1))

0
TERMINAL
Accuracy :0.515
oldold:0.692
old:0.696
new:0.698
----------------------------------------
Accuracy:0.516
Loss:0.700
0.25
TERMINAL
Accuracy :0.511
oldold:0.691
old:0.695
new:0.700
----------------------------------------
Accuracy:0.515
Loss:0.701
0.75
TERMINAL
Accuracy :0.512
oldold:0.722
old:0.783
new:0.785
----------------------------------------
Accuracy:0.494
Loss:0.803


In [14]:
robustest(Dropout=0.5, BN=False, Xavier=True, Activation='LReLU', Pool=(2,1), Filter=(5,3), Dilation=(2,1), Stride=(3,1))

TERMINAL
Accuracy :0.493
oldold:0.701
old:0.705
new:0.715
----------------------------------------
Accuracy:0.509
Loss:0.712


In [15]:
robustest(Dropout=0.5, BN=True, Xavier=False, Activation='LReLU', Pool=(2,1), Filter=(5,3), Dilation=(2,1), Stride=(3,1))

TERMINAL
Accuracy :0.516
oldold:0.697
old:0.705
new:0.705
----------------------------------------
Accuracy:0.499
Loss:0.711


In [17]:
robustest(Dropout=0.5, BN=True, Xavier=True, Activation='ReLU', Pool=(2,1), Filter=(5,3), Dilation=(2,1), Stride=(3,1))

TERMINAL
Accuracy :0.517
oldold:0.696
old:0.700
new:0.706
----------------------------------------
Accuracy:0.496
Loss:0.717
